In [17]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [18]:
house_dict = []

for page in range(16, 21):
    linked = 'https://www.zillow.com/ca/townhomes/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-153.847663625,"east":-84.765632375,"south":12.777434491046995,"north":56.013075839104054},"mapZoom":4,"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"filterState":{"isPreMarketPreForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isSingleFamily":{"value":false},"isManufactured":{"value":false},"isCondo":{"value":false},"isMultiFamily":{"value":false},"isLotLand":{"value":false},"doz":{"value":"90"},"isApartment":{"value":false},"isPreMarketForeclosure":{"value":false}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[0].text
            except:
                zestimate = 'null'
            try:
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[1].text
            except:
                rent_zestimate = 'null'
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(3)
            
        except AttributeError as e:
            continue

print(house_dict)

https://www.zillow.com/community/landings-waypoint/2082500938_zpid/
zestimate null
rent null
https://www.zillow.com/community/parkside-at-tarob-court/2078462623_zpid/
zestimate null
rent null
https://www.zillow.com/community/boulevard-downing/2083235827_zpid/
zestimate null
rent null
https://www.zillow.com/community/avella-at-civita/2071905647_zpid/
zestimate null
rent null
https://www.zillow.com/homedetails/5533-Esprit-Way-Dublin-CA-94568/153093918_zpid/
zestimate $976,047
rent $3,500/mo
https://www.zillow.com/homedetails/24707-6th-St-San-Bernardino-CA-92410/2077118469_zpid/
zestimate $370,076
rent null
https://www.zillow.com/community/copperleaf-at-ironridge/2085330665_zpid/
zestimate null
rent null
https://www.zillow.com/community/portico/2076758016_zpid/
zestimate null
rent null
https://www.zillow.com/community/barcelona/2084312588_zpid/
zestimate null
rent null
https://www.zillow.com/community/pacific-village-pavilion/2082444743_zpid/
zestimate null
rent null
https://www.zillow.co

zestimate null
rent null
https://www.zillow.com/community/foothill-grove/2076663861_zpid/
zestimate null
rent null
https://www.zillow.com/community/park-place/2077730690_zpid/
zestimate null
rent null
https://www.zillow.com/community/westerly/2089228919_zpid/
zestimate null
rent null
https://www.zillow.com/community/innovation-revo/2087796340_zpid/
zestimate null
rent null
https://www.zillow.com/community/laurel-street-townhomes/2071714433_zpid/
zestimate $2,576,735
rent null
https://www.zillow.com/community/foster-square/2088647540_zpid/
zestimate null
rent null
https://www.zillow.com/community/estancia/2080747472_zpid/
zestimate null
rent null
https://www.zillow.com/community/nuevo/2087728953_zpid/
zestimate null
rent null
https://www.zillow.com/community/atherton-place/2076951132_zpid/
zestimate null
rent null
https://www.zillow.com/community/auburn-grove/2073668004_zpid/
zestimate null
rent null
https://www.zillow.com/community/preserve-ridgeview/2077849515_zpid/
zestimate null
ren

zestimate $1,534,789
rent $4,980/mo
https://www.zillow.com/homedetails/729-Cory-Dr-APT-3-Inglewood-CA-90302/82881280_zpid/
zestimate $734,944
rent $3,300/mo
https://www.zillow.com/homedetails/365-Whispering-Willow-Dr-UNIT-D-Santee-CA-92071/16884651_zpid/
zestimate $548,123
rent $2,500/mo
https://www.zillow.com/homedetails/425-Stratus-Ln-UNIT-6-Simi-Valley-CA-93065/131677593_zpid/
zestimate $630,261
rent $3,250/mo
https://www.zillow.com/homedetails/2142-Vista-Dorado-Newport-Beach-CA-92660/25479230_zpid/
zestimate $1,749,001
rent $4,776/mo
https://www.zillow.com/homedetails/7401-S-Sepulveda-Blvd-APT-113-Los-Angeles-CA-90045/114417021_zpid/
zestimate $1,738,631
rent $6,500/mo
https://www.zillow.com/homedetails/6201-Valley-Circle-Blvd-UNIT-9-Los-Angeles-CA-91367/19872756_zpid/
zestimate $835,099
rent $3,500/mo
https://www.zillow.com/homedetails/11190-Caminito-Inocenta-San-Diego-CA-92126/16779273_zpid/
zestimate $923,332
rent $3,400/mo
https://www.zillow.com/homedetails/30389-Town-Square-Dr

In [19]:
house_data = pd.DataFrame(house_dict)
house_data

,Listing_type,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zestimate,Rent_zestimate,Zillow_price/sqft,House_website
0,,"$545,990+","Residence Three Plan, The Landings : Waypoint",Residence Three Plan,TheLandings:Waypoint,null,null,3,3,"1,668",No Data,null,Townhouse,2021,No Data,null,null,null,$327,https://www.zillow.com/community/landings-wayp...
1,,"$1,387,995+","Sage Plan, Parkside at Tarob Court",Sage Plan,ParksideatTarobCourt,null,null,3,3,"1,849",No Data,null,Townhouse,2021,No Data,null,null,null,$751,https://www.zillow.com/community/parkside-at-t...
2,,"$1,035,880+","Residence Four Plan, Boulevard : Downing",Residence Four Plan,Boulevard:Downing,null,null,4,3,"2,492",No Data,null,Townhouse,2021,No Data,null,null,null,$416,https://www.zillow.com/community/boulevard-dow...
3,,"$1,341,995+","A-4 Plan, Avella at Civita",A-4 Plan,AvellaatCivita,null,null,3,4,"2,346",No Data,null,Townhouse,2021,No Data,null,null,null,$572,https://www.zillow.com/community/avella-at-civ...
4,,"$890,000","5533 Esprit Way, Dublin, CA 94568",5533 Esprit Way,Dublin,CA,94568,4,3,"1,753",No Data,null,Townhouse,2015,"Garage Door Opener, Attached",$189 monthly,"$976,047","$3,500/mo",$508,https://www.zillow.com/homedetails/5533-Esprit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,,"$988,000","515 King Dr APT 1, Daly City, CA 94015",515 King Dr APT 1,DalyCity,CA,94015,3,4,"1,610",696 sqft,696,Townhouse,1982,Carport,$388 monthly,"$1,019,679","$3,500/mo",$614,https://www.zillow.com/homedetails/515-King-Dr...
196,,"$1,390,000","2 Lisbon Ln, Redwood City, CA 94063",2 Lisbon Ln,RedwoodCity,CA,94063,3,4,"1,648","1,648 sqft",1648,Townhouse,2021,"Garage Door Opener, Attached, Side By Side",$399 monthly,"$1,443,543",null,$843,https://www.zillow.com/homedetails/2-Lisbon-Ln...
197,,"$1,050,000","7 Lisbon Ln, Redwood City, CA 94063",7 Lisbon Ln,RedwoodCity,CA,94063,2,2,"1,150","1,150 sqft",1150,Townhouse,2021,"Garage Door Opener, Attached, Tandem",$399 monthly,"$1,074,034",null,$913,https://www.zillow.com/homedetails/7-Lisbon-Ln...
198,DOUGLAS ELLIMAN,"$2,479,000","2215 5th St #A, Santa Monica, CA 90405",2215 5th St #A,SantaMonica,CA,90405,3,3,"2,200",No Data,null,Townhouse,2020,"Attached Carport, Gated, Side By Side, Covered...",$250 monthly,"$2,529,216",null,"$1,127",https://www.zillow.com/homedetails/2215-5th-St...


In [20]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q1-2021_Townhouse_Data_page16-20.csv", index=False, header=True)